In [2]:
import tensorflow as tf
import numpy as np
import alpaca_trade_api as tradeapi
from sklearn.model_selection import train_test_split
import neptune
import pandas as pd
from fredapi import Fred

In [19]:
df = pd.read_csv('stock_data_full_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)
df['timestamp'] = df['timestamp'].dt.date
df.set_index('timestamp', inplace=True)

In [20]:
df_bin1 = pd.read_csv('bin1_stocks.csv')
df_bin2 = pd.read_csv('bin2_stocks.csv')
df_bin3 = pd.read_csv('bin3_stocks.csv')
df_bin4 = pd.read_csv('bin4_stocks.csv')
df_bin5 = pd.read_csv('bin5_stocks.csv')

In [21]:
bins = [df_bin1, df_bin2, df_bin3, df_bin4, df_bin5]

In [22]:
for bin in bins:
    bin.drop(columns=['Unnamed: 0'], inplace=True)

In [23]:
# Prepare (mostly macro-economic) Features that are used by every stock or ETF

# Prepare macro-economic features
fred_api_key = 'b0fae0f02614161be351f0a9d3517207'
fred = Fred(api_key=fred_api_key)
start_date = '2020-05-08'
end_date = '2025-05-07'

indicators = {
    "real_GDP": "GDPC1",
    "unemployment_rate": "UNRATE",
    "CPI": "CPIAUCSL",
    "personal_consumption": "PCE",
    "industrial_production": "INDPRO",
    "federal_funds_rate": "FEDFUNDS",
    "10Y_treasury_rate": "DGS10",
    "retail_sales": "RSAFS",
    "housing_starts": "HOUST",
    "usd_eur": "DEXUSEU",
    "usd_gbp": "DEXUSUK",
    "usd_jpy": "DEXJPUS"
}

# iterate to get data for each indicator
data = {}
for name, series_id in indicators.items():
    print(f"Downloading {name}...")
    series_data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
    series_data = series_data.to_frame(name=name)
    data[name] = series_data

# Combine all macro-economic indicators into a single DataFrame
df_macro = pd.concat(data.values(), axis=1)
df_macro = df_macro.ffill()
df_macro = df_macro[df_macro.index >= '2020-05-08']
df_macro.to_csv("macro_data.csv", index=True)

In [75]:
chosen_stocks = []
for bin in bins:
    df_bin = []
    for ticker in bin.columns:
        df_ticker = df[df['symbol'] == ticker].copy()


        # ---
        # Add Technical indicators
        # ---


        # Add Relative Strength Index (RSI)
        window = 14
        delta = df_ticker['close'].diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.ewm(alpha=1/window, min_periods=window).mean()
        avg_loss = loss.ewm(alpha=1/window, min_periods=window).mean() 
        rs = avg_gain / avg_loss
        df_ticker['rsi'] = 100 - (100 / (1 + rs))

        # Add Rate of Change (ROC)
        window1 = 9
        window2 = 14
        window3 = 21
        prev_price1 = df_ticker['close'].shift(window1)
        df_ticker['roc_9'] = (df_ticker['close'] - prev_price1) / prev_price1 * 100
        prev_price2 = df_ticker['close'].shift(window2)
        df_ticker['roc_14'] = (df_ticker['close'] - prev_price2) / prev_price2 * 100
        prev_price3 = df_ticker['close'].shift(window3)
        df_ticker['roc_21'] = (df_ticker['close'] - prev_price3) / prev_price3 * 100

        # Add Moving Average Convergence Divergence (MACD)
        short_span = 12
        long_span = 26
        signal_span = 9
        ema_short = df_ticker['close'].ewm(span=short_span, adjust=False).mean()
        ema_long = df_ticker['close'].ewm(span=long_span, adjust=False).mean()
        df_ticker['macd'] = ema_short - ema_long
        df_ticker['macd_signal'] = df_ticker['macd'].ewm(span=signal_span, adjust=False).mean()
        df_ticker['macd_hist'] = df_ticker['macd'] - df_ticker['macd_signal']

        # Add Stochastic Oscillator
        window_K = 14 # for %K
        window_D = 3
        highest_high = df_ticker['high'].rolling(window=window_K).max()
        lowest_low = df_ticker['low'].rolling(window=window_K).min()
        df_ticker['stoch_%K'] = (df_ticker['close'] - lowest_low) / (highest_high - lowest_low) * 100
        df_ticker['stoch_%D'] = df_ticker['stoch_%K'].rolling(window=window_D).mean()
        df_ticker['stoch_diff'] = df_ticker['stoch_%K'] - df_ticker['stoch_%D']

        # Add Commodity Channel Index (CCI)
        window = 20
        typical_price = (df_ticker['high'] + df_ticker['low'] + df_ticker['close']) / 3
        sma = typical_price.rolling(window=window, min_periods=int(window/2)).mean()
        mad = (typical_price - sma).abs().rolling(window=window).mean()
        df_ticker['cci'] = (typical_price - sma) / (0.015 * mad)


        # ---
        # Add Trend Indicators
        # ---


        # Add different Moving Averages
        df_ticker['ma_15'] = df_ticker['close'].rolling(window=15, min_periods=5).mean()
        df_ticker['ma_30'] = df_ticker['close'].rolling(window=30, min_periods=10).mean()
        df_ticker['ma_60'] = df_ticker['close'].rolling(window=60, min_periods=15).mean()
        df_ticker['ma_100'] = df_ticker['close'].rolling(window=100, min_periods=20).mean()
        df_ticker['ma_200'] = df_ticker['close'].rolling(window=200, min_periods=30).mean()



        # Add Average True Range (ATR)
        window = 14
        high_low = df_ticker['high'] - df_ticker['low']
        high_close = (df_ticker['high'] - df_ticker['close'].shift()).abs()
        low_close = (df_ticker['low'] - df_ticker['close'].shift()).abs()
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df_ticker['atr'] = tr.rolling(window=window).mean()
        
        # Add Average Directional Index (ADX)
        up_move = df_ticker['high'] - df_ticker['high'].shift()
        down_move = df_ticker['low'].shift() - df_ticker['low']
        plus_dm = pd.Series(np.where((up_move > down_move) & (up_move > 0), up_move, 0), index=df_ticker.index)
        minus_dm = pd.Series(np.where((down_move > up_move) & (down_move > 0), down_move, 0), index=df_ticker.index)
        window_dx = 14
        df_ticker['plus_di'] = 100 * (plus_dm.rolling(window=window_dx).mean() / df_ticker['atr'])
        df_ticker['minus_di'] = 100 * (minus_dm.rolling(window=window_dx).mean() / df_ticker['atr'])
        df_ticker['dx'] = 100 * (abs(df_ticker['plus_di'] - df_ticker['minus_di']) / (df_ticker['plus_di'] + df_ticker['minus_di']))


        # Calculate ADX
        window = 14
        dx_rolling = df_ticker['dx'].rolling(window=window).mean()
        first_valid_index = dx_rolling.first_valid_index()
        df_ticker['adx'] = np.nan
        df_ticker.loc[first_valid_index, 'adx'] = dx_rolling[first_valid_index]
        first_valid_index = df_ticker.index.get_loc(first_valid_index)
        for i in range(first_valid_index + 1, len(df_ticker)):
            df_ticker.loc[df_ticker.index[i], 'adx'] = (
                (df_ticker.loc[df_ticker.index[i - 1], 'adx'] * (window - 1) + df_ticker.loc[df_ticker.index[i], 'dx']) / window
            )


        # ---
        # Add Volatility Indicator
        # ---


        # Add Bollinger Bands
        window = 20
        df_ticker['bb_middle'] = df_ticker['close'].rolling(window=window).mean()
        df_ticker['bb_std'] = df_ticker['close'].rolling(window=window).std()
        df_ticker['bb_upper'] = df_ticker['bb_middle'] + (df_ticker['bb_std'] * 2)
        df_ticker['bb_lower'] = df_ticker['bb_middle'] - (df_ticker['bb_std'] * 2)
        df_ticker['bb_pct'] = (df_ticker['close'] - df_ticker['bb_lower']) / (df_ticker['bb_upper'] - df_ticker['bb_lower'])
        df_ticker['bb_width'] = (df_ticker['bb_upper'] - df_ticker['bb_lower']) / df_ticker['bb_middle']
        df_ticker['bb_d_to_upper'] = df_ticker['close'] - df_ticker['bb_upper']
        df_ticker['bb_d_to_lower'] = df_ticker['close'] - df_ticker['bb_lower']


        # ---
        # Add Volume Indicators
        # ---


        # Add On-Balance Volume (OBV)
        df_ticker['obv'] = 0
        for i in range(1, len(df_ticker)):
            if df_ticker['close'].iloc[i] > df_ticker['close'].iloc[i-1]:
                df_ticker.loc[df_ticker.index[i], 'obv'] = df_ticker['obv'].iloc[i-1] + df_ticker['volume'].iloc[i]
            elif df_ticker['close'].iloc[i] < df_ticker['close'].iloc[i-1]:
                df_ticker.loc[df_ticker.index[i], 'obv'] = df_ticker['obv'].iloc[i-1] - df_ticker['volume'].iloc[i]
            else:
                df_ticker.loc[df_ticker.index[i], 'obv'] = df_ticker['obv'].iloc[i-1]
        df_ticker['obv_norm'] = (df_ticker['obv'] - df_ticker['obv'].min()) / (df_ticker['obv'].max() - df_ticker['obv'].min())
        df_ticker['obv_momentum'] = df_ticker['obv'] - df_ticker['obv'].shift(10)

        # Add Volume Price Trend (VPT)
        df_ticker['vpt'] = 0
        vpt_change = df_ticker['close'].pct_change(fill_method=None) * df_ticker['volume']
        df_ticker['vpt'] = df_ticker['vpt'].shift() + vpt_change
        df_ticker['vpt_norm'] = (df_ticker['vpt'] - df_ticker['vpt'].min()) / (df_ticker['vpt'].max() - df_ticker['vpt'].min())
        df_ticker['vpt_momentum'] = df_ticker['vpt'] - df_ticker['vpt'].shift(10)


        # ---
        # Add Macro-Economic Indicators
        # ---


        # Add S&P 500 Index
        df_ticker['sp500'] = df[df['symbol'] == 'SPY']['close'].copy()

        # Add calendaric features
        df_ticker.reset_index(inplace=True)
        df_ticker.set_index('timestamp', inplace=True)
        df_ticker.index = pd.to_datetime(df_ticker.index)
        df_ticker.index = df_ticker.index.date
        df_ticker.index = pd.to_datetime(df_ticker.index)
        df_ticker['dayofweek'] = df_ticker.index.day_of_week
        df_ticker['dayofyear'] = df_ticker.index.day_of_year
        df_ticker['month'] = df_ticker.index.month
        df_ticker['year'] = df_ticker.index.year
        df_ticker['quarter'] = df_ticker.index.quarter

        # Add macro-economic indicators that were prepared earlier in the pipeline
        df_ticker = df_ticker.join(df_macro, how='left')

        # Add the target variable which is the next day's close price
        df_ticker['target'] = df_ticker['close'].shift(-1)
        
        df_bin.append(df_ticker.copy())

    chosen_stocks.append(df_bin.copy())

In [76]:
for column in chosen_stocks[1][14].columns:
    print(f"{column}: {chosen_stocks[1][14][column].isna().sum()}")

close: 0
high: 0
low: 0
trade_count: 0
open: 0
volume: 0
vwap: 0
symbol: 0
rsi: 14
roc_9: 9
roc_14: 14
roc_21: 21
macd: 0
macd_signal: 0
macd_hist: 0
stoch_%K: 13
stoch_%D: 15
stoch_diff: 15
cci: 28
ma_15: 4
ma_30: 9
ma_60: 14
ma_100: 19
ma_200: 29
atr: 13
plus_di: 13
minus_di: 13
dx: 13
adx: 26
bb_middle: 19
bb_std: 19
bb_upper: 19
bb_lower: 19
bb_pct: 19
bb_width: 19
bb_d_to_upper: 19
bb_d_to_lower: 19
obv: 0
obv_norm: 0
obv_momentum: 10
vpt: 1
vpt_norm: 1
vpt_momentum: 11
sp500: 0
dayofweek: 0
dayofyear: 0
month: 0
year: 0
quarter: 0
real_GDP: 0
unemployment_rate: 0
CPI: 0
personal_consumption: 0
industrial_production: 0
federal_funds_rate: 0
10Y_treasury_rate: 0
retail_sales: 0
housing_starts: 0
usd_eur: 0
usd_gbp: 0
usd_jpy: 0
target: 1


In [77]:
chosen_stocks[1][14]

,close,high,low,trade_count,open,volume,vwap,symbol,rsi,roc_9,...,personal_consumption,industrial_production,federal_funds_rate,10Y_treasury_rate,retail_sales,housing_starts,usd_eur,usd_gbp,usd_jpy,target
2020-05-08,15.200,15.2200,15.1400,27,15.2100,4499,15.170000,HTUS,NaN,NaN,...,13251.7,86.0108,0.05,0.69,478449.0,1039.0,1.0854,1.2436,106.50,15.57
2020-05-11,15.570,15.5700,15.1800,29,15.2000,3760,15.270000,HTUS,NaN,NaN,...,13251.7,86.0108,0.05,0.73,478449.0,1039.0,1.0818,1.2330,107.70,15.57
2020-05-12,15.570,15.5700,15.5700,0,15.5700,0,0.000000,HTUS,NaN,NaN,...,13251.7,86.0108,0.05,0.69,478449.0,1039.0,1.0864,1.2299,107.33,15.57
2020-05-13,15.570,15.5700,15.5700,0,15.5700,0,0.000000,HTUS,NaN,NaN,...,13251.7,86.0108,0.05,0.64,478449.0,1039.0,1.0837,1.2225,106.92,15.57
2020-05-14,15.570,15.5700,15.5700,0,15.5700,0,0.000000,HTUS,NaN,NaN,...,13251.7,86.0108,0.05,0.63,478449.0,1039.0,1.0800,1.2194,107.09,15.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01,36.765,36.9350,36.6200,71,36.6400,5232,36.701574,HTUS,57.723977,6.848598,...,20669.5,103.8799,4.33,4.25,724131.0,1361.0,1.1279,1.3273,145.48,37.02
2025-05-02,37.020,37.0200,36.8412,67,36.8412,4978,36.925397,HTUS,59.059653,9.982175,...,20669.5,103.8799,4.33,4.33,724131.0,1361.0,1.1330,1.3283,144.52,36.73
2025-05-05,36.730,36.9600,36.7300,85,36.7900,24237,36.845961,HTUS,56.859501,4.966550,...,20669.5,103.8799,4.33,4.36,724131.0,1361.0,1.1315,1.3284,143.98,36.56
2025-05-06,36.560,36.6599,36.4700,60,36.5290,6397,36.587488,HTUS,55.553015,3.657499,...,20669.5,103.8799,4.33,4.30,724131.0,1361.0,1.1345,1.3369,142.76,36.57


In [78]:
for i in range(len(chosen_stocks)):
    print(f"Bin {i+1} contains {len(chosen_stocks[i])} stocks.")
    for j in range(len(chosen_stocks[i])):
        print(f"Date range: {chosen_stocks[i][j].index.min().date()} - {chosen_stocks[i][j].index.max().date()}")
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j])} rows.")
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j].columns)} columns.")
        print(f"Stock {j+1} in Bin {i+1} has missing values: {chosen_stocks[i][j].isnull().sum().sum()}")

Bin 1 contains 15 stocks.
Date range: 2020-05-08 - 2025-05-07
Stock 1 in Bin 1 contains 1256 rows.
Stock 1 in Bin 1 contains 62 columns.
Stock 1 in Bin 1 has missing values: 458
Date range: 2020-05-08 - 2025-05-07
Stock 2 in Bin 1 contains 1256 rows.
Stock 2 in Bin 1 contains 62 columns.
Stock 2 in Bin 1 has missing values: 458
Date range: 2020-05-08 - 2025-05-07
Stock 3 in Bin 1 contains 1256 rows.
Stock 3 in Bin 1 contains 62 columns.
Stock 3 in Bin 1 has missing values: 458
Date range: 2020-05-08 - 2025-05-07
Stock 4 in Bin 1 contains 1256 rows.
Stock 4 in Bin 1 contains 62 columns.
Stock 4 in Bin 1 has missing values: 458
Date range: 2020-05-08 - 2025-05-07
Stock 5 in Bin 1 contains 1256 rows.
Stock 5 in Bin 1 contains 62 columns.
Stock 5 in Bin 1 has missing values: 458
Date range: 2020-05-08 - 2025-05-07
Stock 6 in Bin 1 contains 1256 rows.
Stock 6 in Bin 1 contains 62 columns.
Stock 6 in Bin 1 has missing values: 458
Date range: 2020-05-08 - 2025-05-07
Stock 7 in Bin 1 contains 

In [65]:
copy = chosen_stocks.copy()

In [79]:
for i in range(len(chosen_stocks)):
    print(f"Bin {i+1} contains {len(chosen_stocks[i])} stocks.")
    for j in range(len(chosen_stocks[i])):
        chosen_stocks[i][j].dropna(inplace=True)
        print(f"After dropping NaN values:")
        print(f"Date range: {chosen_stocks[i][j].index.min().date()} - {chosen_stocks[i][j].index.max().date()}")
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j])} rows.")
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j].columns)} columns.")
        print(f"Stock {j+1} in Bin {i+1} has missing values: {chosen_stocks[i][j].isnull().sum().sum()}")

Bin 1 contains 15 stocks.
After dropping NaN values:
Date range: 2020-06-19 - 2025-05-06
Stock 1 in Bin 1 contains 1226 rows.
Stock 1 in Bin 1 contains 62 columns.
Stock 1 in Bin 1 has missing values: 0
After dropping NaN values:
Date range: 2020-06-19 - 2025-05-06
Stock 2 in Bin 1 contains 1226 rows.
Stock 2 in Bin 1 contains 62 columns.
Stock 2 in Bin 1 has missing values: 0
After dropping NaN values:
Date range: 2020-06-19 - 2025-05-06
Stock 3 in Bin 1 contains 1226 rows.
Stock 3 in Bin 1 contains 62 columns.
Stock 3 in Bin 1 has missing values: 0
After dropping NaN values:
Date range: 2020-06-19 - 2025-05-06
Stock 4 in Bin 1 contains 1226 rows.
Stock 4 in Bin 1 contains 62 columns.
Stock 4 in Bin 1 has missing values: 0
After dropping NaN values:
Date range: 2020-06-19 - 2025-05-06
Stock 5 in Bin 1 contains 1226 rows.
Stock 5 in Bin 1 contains 62 columns.
Stock 5 in Bin 1 has missing values: 0
After dropping NaN values:
Date range: 2020-06-19 - 2025-05-06
Stock 6 in Bin 1 contains 

In [80]:
for i in range(len(chosen_stocks)):
    for j in range(len(chosen_stocks[i])):
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j])} rows.")
        

Stock 1 in Bin 1 contains 1226 rows.
Stock 2 in Bin 1 contains 1226 rows.
Stock 3 in Bin 1 contains 1226 rows.
Stock 4 in Bin 1 contains 1226 rows.
Stock 5 in Bin 1 contains 1226 rows.
Stock 6 in Bin 1 contains 1226 rows.
Stock 7 in Bin 1 contains 1226 rows.
Stock 8 in Bin 1 contains 1226 rows.
Stock 9 in Bin 1 contains 1226 rows.
Stock 10 in Bin 1 contains 1226 rows.
Stock 11 in Bin 1 contains 1226 rows.
Stock 12 in Bin 1 contains 1226 rows.
Stock 13 in Bin 1 contains 1226 rows.
Stock 14 in Bin 1 contains 1226 rows.
Stock 15 in Bin 1 contains 1226 rows.
Stock 1 in Bin 2 contains 1226 rows.
Stock 2 in Bin 2 contains 1226 rows.
Stock 3 in Bin 2 contains 1226 rows.
Stock 4 in Bin 2 contains 1226 rows.
Stock 5 in Bin 2 contains 1226 rows.
Stock 6 in Bin 2 contains 1226 rows.
Stock 7 in Bin 2 contains 1226 rows.
Stock 8 in Bin 2 contains 1226 rows.
Stock 9 in Bin 2 contains 1226 rows.
Stock 10 in Bin 2 contains 1226 rows.
Stock 11 in Bin 2 contains 1226 rows.
Stock 12 in Bin 2 contains 122

In [94]:
import pickle
with open('processed_data/preprocessed_general.pkl', 'wb') as f:
    pickle.dump(chosen_stocks, f)

In [92]:
chosen_stocks[0][0].columns

Index(['close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap',
       'symbol', 'rsi', 'roc_9', 'roc_14', 'roc_21', 'macd', 'macd_signal',
       'macd_hist', 'stoch_%K', 'stoch_%D', 'stoch_diff', 'cci', 'ma_15',
       'ma_30', 'ma_60', 'ma_100', 'ma_200', 'atr', 'plus_di', 'minus_di',
       'dx', 'adx', 'bb_middle', 'bb_std', 'bb_upper', 'bb_lower', 'bb_pct',
       'bb_width', 'bb_d_to_upper', 'bb_d_to_lower', 'obv', 'obv_norm',
       'obv_momentum', 'vpt', 'vpt_norm', 'vpt_momentum', 'sp500', 'dayofweek',
       'dayofyear', 'month', 'year', 'quarter', 'real_GDP',
       'unemployment_rate', 'CPI', 'personal_consumption',
       'industrial_production', 'federal_funds_rate', '10Y_treasury_rate',
       'retail_sales', 'housing_starts', 'usd_eur', 'usd_gbp', 'usd_jpy',
       'target'],
      dtype='object')